# Executive Summary:

### Brief Problem Overview:

Trojan E-commerce has 17 fulfillment centers across the US and this year they have the capital to implement small scale capacity increases at a few of the fulfillment centers. They would like to identify the top five fulfilment centers in which a small-scale capacity expansion would yield the greatest cost savings to Trojan’s supply chain. In the data given, the items Trojan offers have been clustered into 100 representative items, the US has been split into 98 demand regions, and the weekly demand of each region has been estimated. Trojan wishes to minimize the total transportation cost but are constrained by the capacity at each fulfillment center, and of course satisfying the weekly demand in each region.

### The minimum total shipping cost for "data.xlsx":
The minimum total shipping cost for Nia's company calculated from the information given in "data.xlsx" is $9,841,229.29

### Recommended FCs to invest in a small scale capacity increase so as to achieve the highest cost savings:

The recommended FC's to invest in a small scale capacity increase are as follows: <br>
- TPA1 with a shadow price of -5.333088
- MKE1 with a shadow price of -4.276396
- SDF8 with a shadow price of -2.486980
- BDL1 with a shadow price of -2.302010
- EWR4 with a shadow price of -1.807986

# Abstract Formulation

**Data:** 

- $I$: the set of fulfillment centers (FC's). 
- $J$: the set of demand regions.
- $K$: the set of items
- $w_k$: the shipping weight of item $k$.
- $𝛿_{ij}$: the the distance from FC $i$ to region $j$ (thousands of miles).
- $d_{jk}$: weekly demand for item $k$ in region $j$.
- $s_k$: the amount of space required to store one unit of item $k$.
- $q_i$: total capacity of FC $i$.

**Decision Variables:** For each FC $i \in I$ , each demand region $j \in J$, and each item $k \in K$, let $x_{ijk}$ denote the number of units to send of item $k$ from FC $i$ to demand region $j$. (Integer)

**Objective and constraints:**

$$\begin{aligned}
\text{Minimize:} && \sum_{i \in I, j\in J, k\in K} 1.38w_k𝛿_{ij}x_{ijk} \\
\text{subject to:} \\
\text{(FC Capacity)} && \sum_{j \in J,k \in K} x_{ijk}s_k & \le q_i && \text{ for $i \in I$.}\\
\text{(Demand)} && \sum_{i \in I} x_{ijk} & \ge d_{jk} && \text{ for $j \in J$, $k \in K$.}\\
\text{(Non-Negativity)} && x_{ijk} & \ge 0 && \text{for $i \in I, j \in J, k \in K$} \\
\end{aligned}$$

# Technical Appendix

In [29]:
#importing necessary packages, importing data

import pandas as pd
from gurobipy import Model, GRB

fcs = pd.read_excel('small_data.xlsx',index_col=0) #fcs
regions = pd.read_excel('small_data.xlsx',sheet_name = 'Regions',index_col=0) #regions
distance = pd.read_excel('small_data.xlsx',sheet_name = 'Distances',index_col=0)
items = pd.read_excel('small_data.xlsx',sheet_name = 'Items',index_col=0) #items
demand = pd.read_excel('small_data.xlsx',sheet_name = 'Demand',index_col=0) #item

In [30]:
#setting up data variables
I = fcs.index                #set of fcs
J = regions.index            #set of regions
K = items.index              #set of items  
w = items['shipping_weight'] #item shipping weight              w_k
q = fcs['capacity']          #fc capacity                       q_i
s = items['storage_size']    #item storage size                 s_k
# distance.loc[j,i]          #distance from fc i to region j    𝛿_ij
# demand.loc[k,j]            #demand for item k in region j     d_jk

In [31]:
#initial optimization model for smalldata
mod = Model()

x = mod.addVars(I,J,K)
mod.setObjective(sum(1.38*w[k]*distance.loc[j,i]*x[i,j,k] for i in I for j in J for k in K))
fc_constraint = {}
for i in I:
    fc_constraint[i]=mod.addConstr(sum(x[i,j,k]*s[k] for j in J for k in K)<=q[i])
for k in K:
    for j in J:
        mod.addConstr(sum(x[i,j,k] for i in I)>=demand.loc[k,j])
mod.setParam('outputflag',False)
mod.optimize()
mod.objval

3400.769189999999

In [32]:
#adding in excel writer

mod = Model()

x = mod.addVars(I,J,K)
mod.setObjective(sum(1.38*w[k]*distance.loc[j,i]*x[i,j,k] for i in I for j in J for k in K))
fc_constraint = {}
for i in I:
    fc_constraint[i]=mod.addConstr(sum(x[i,j,k]*s[k] for j in J for k in K)<=q[i])
for k in K:
    for j in J:
        mod.addConstr(sum(x[i,j,k] for i in I)>=demand.loc[k,j])
mod.setParam('outputflag',False)
mod.optimize()
writer=pd.ExcelWriter('smalldata.xlsx')
pd.DataFrame([mod.objVal],columns=['Objective Value']).to_excel(writer,sheet_name='Summary',index=False)
df = pd.DataFrame(columns=['FC_name','region_ID','item_ID','shipment'])
for i in I:
    for j in J:
        for k in K:
            if x[i,j,k].x > 0:
                newrow = {'FC_name':i,'region_ID':j,'item_ID':k,'shipment':x[i,j,k].x}
                df = df.append(newrow,True)
df.to_excel(writer,sheet_name='Solution',index=False)
shadow_prices = pd.DataFrame(columns=['FC_name', 'shadow_price'])
for key, value in fc_constraint.items():
        shadow_prices = shadow_prices.append({'FC_name': key, 'shadow_price': value.pi}, True)
shadow_prices = shadow_prices.sort_values(by = 'shadow_price').reset_index(drop = True).loc[:4, :]
shadow_prices.to_excel(writer, sheet_name='Capacity Constraints', index=False)
writer.save()

In [33]:
#creating function
def optimize(inputFile,outputFile):
    '''This function will take an excel sheet in the same format as 'data.xlsx' as an input and requires a desired
    file path for the output file. It will return what is the minimum transportation cost achievable with the 
    input data. It will also return the 5 fulfillment centers with the most negative shadow prices. These 5 fulfillment
    centers are the recommended FCs to make small scale capacity increases at in order to lower the minimum 
    transportation cost. ''' 
    import pandas as pd
    from gurobipy import Model, GRB
    fcs = pd.read_excel(inputFile,index_col=0) 
    regions = pd.read_excel(inputFile,sheet_name = 'Regions',index_col=0) 
    distance = pd.read_excel(inputFile,sheet_name = 'Distances',index_col=0)
    items = pd.read_excel(inputFile,sheet_name = 'Items',index_col=0) 
    demand = pd.read_excel(inputFile,sheet_name = 'Demand',index_col=0) 
    I = fcs.index                
    J = regions.index            
    K = items.index              
    w = items['shipping_weight']              
    q = fcs['capacity']          
    s = items['storage_size']    
    mod = Model()
    x = mod.addVars(I,J,K)
    mod.setObjective(sum(1.38*w[k]*distance.loc[j,i]*x[i,j,k] for i in I for j in J for k in K))
    fc_constraint = {}
    for i in I:
        fc_constraint[i]=mod.addConstr(sum(x[i,j,k]*s[k] for j in J for k in K)<=q[i])
    for k in K:
        for j in J:
            mod.addConstr(sum(x[i,j,k] for i in I)>=demand.loc[k,j])
    mod.setParam('outputflag',False)
    mod.optimize()
    writer=pd.ExcelWriter(outputFile)
    pd.DataFrame([mod.objVal],columns=['Objective Value']).to_excel(writer,sheet_name='Summary',index=False)
    df = pd.DataFrame(columns=['FC_name','region_ID','item_ID','shipment'])
    for i in I:
        for j in J:
            for k in K:
                if x[i,j,k].x > 0:
                    newrow = {'FC_name':i,'region_ID':j,'item_ID':k,'shipment':x[i,j,k].x}
                    df = df.append(newrow,True)
    df.to_excel(writer,sheet_name='Solution',index=False)
    shadow_prices = pd.DataFrame(columns=['FC_name', 'shadow_price'])
    for key, value in fc_constraint.items():
            shadow_prices = shadow_prices.append({'FC_name': key, 'shadow_price': value.pi}, True)
    shadow_prices = shadow_prices.sort_values(by = 'shadow_price').reset_index(drop = True).loc[:4, :]
    shadow_prices.to_excel(writer, sheet_name='Capacity Constraints', index=False)
    writer.save()   

In [34]:
#testing function

optimize('data.xlsx','data_results.xlsx')

In [35]:
#reading results of function

min_cost= pd.read_excel('data_results.xlsx',sheet_name='Summary',index_col=0)
top5fcs = pd.read_excel('data_results.xlsx',sheet_name='Capacity Constraints',index_col=0)
print(min_cost,top5fcs)

Empty DataFrame
Columns: []
Index: [9841229.288170155]          shadow_price
FC_name              
TPA1        -5.333088
MKE1        -4.276396
SDF8        -2.486980
BDL1        -2.302010
EWR4        -1.807986


In [ ]:
#code to add at end of function, only going to add to .py file

if __name__=='__main__':
    import sys, os
    if len(sys.argv)!=3:
        print('Correct syntax: python lab2_code.py inputFile outputFile')
    else:
        inputFile=sys.argv[1]
        outputFile=sys.argv[2]
        if os.path.exists(inputFile):
            optimize(inputFile,outputFile)
            print(f'Successfully optimized. Results in "{outputFile}"')
        else:
            print(f'File "{inputFile}" not found!')